In [8]:
from ipywidgets import widgets, HBox, VBox, Layout
from IPython.display import display
from functools import partial
import numpy as np
#import tictactoe
import tictactoe_minimax_helper as minimax_helper

In [9]:
class General_functions(object):
    def __init__(self, matrix, actual_turn):
        self.N = 3
        self.button_list = None
        self.text_box = None
        self.matrix = matrix
        self.game_finished = False
        self.actual_turn = actual_turn
        
    def display_matrix(self):
        N = self.N
        childs = []
        for i in range(N):
            for j in range(N):
                if self.matrix[i,j] == 1:
                    self.button_list[i * N + j].description = 'o'
                if self.matrix[i,j] == -1:
                    self.button_list[i * N + j].description = 'x'

    def on_button_clicked(self, index, button):
        N = self.N  

        if self.game_finished:
            return

        y = index % N
        x = int(index / N)
        if self.matrix[x,y] != 0:
            self.text_box.value = '거긴 못 놓음ㅋ'
            return
        button.description = self.actual_turn[0]

        if self.actual_turn == 'o':
            self.matrix[x,y] = 1
            self.game_finished, status = minimax_helper.game_over(self.matrix)
            if self.game_finished:
                if (status != 0):
                    self.text_box.value = 'o 승'
                else: 
                    self.text_box.value = '무승부'
            else:
                self.actual_turn = 'x'
                self.text_box.value = self.actual_turn + ' 차례' 
        else:
            self.matrix[x,y] = -1
            self.game_finished, status = minimax_helper.game_over(self.matrix)
            if self.game_finished:
                if (status != 0):
                    self.text_box.value = 'x 승'
                else: 
                    self.text_box.value = '무승부'
            else:
                self.actual_turn = 'o'
                self.text_box.value = self.actual_turn + ' 차례' 
        self.computer_play()
        
    def draw_board(self):
        self.text_box = widgets.Text(value = self.actual_turn + ' 차례', layout = Layout(width = '129px', height = '40px'))
        self.button_list = []
        for i in range(9):
            button = widgets.Button(description = '',
            disabled = False,
            button_style = '', # 'success', 'info', 'warning', 'danger' or ''
            tooltip = 'Click me',
            icon = '',
            layout = Layout(width = '40px', height = '40px'))
            self.button_list.append(button)
            button.on_click(partial(self.on_button_clicked, i))
        tic_tac_toe_board = VBox([HBox([self.button_list[0], self.button_list[1], self.button_list[2]]),
                HBox([self.button_list[3], self.button_list[4], self.button_list[5]]),
                HBox([self.button_list[6], self.button_list[7], self.button_list[8]])])
        display(VBox([self.text_box, tic_tac_toe_board]))
        return

    def computer_play(self):

        if self.game_finished:
            return
        
        if self.actual_turn == 'x':
            turn = -1
            next_turn = 'o'
        if self.actual_turn == 'o':
            turn = 1
            next_turn = 'x'
        self.matrix = self.get_best_play(turn)
        self.display_matrix()
        self.actual_turn = next_turn
        self.text_box.value = self.actual_turn + ' 차례'
        self.game_finished, status = minimax_helper.game_over(self.matrix)
        if self.game_finished:
            if (status != 0):
                self.text_box.value = '컴퓨터 승'
            else: 
                self.text_box.value = '무승부'

    def get_best_play(self, turn):
        # 1000 is an infinite value compared with the highest cost of 10 that we can get

        choice, points, nodes_visited = minimax_helper.minimax(self.matrix, turn)
        print('points:',points)
        print('nodes_visited:', nodes_visited)
        return choice

In [10]:
def start_game(computer_starts = True, user_icon = 'x', start_mode = 'center'):
    matrix = np.zeros((3,3))
    
    if user_icon == 'x':
        computer_icon_representation = 1
    else:
        computer_icon_representation = -1

    GF = General_functions(matrix, user_icon)
    GF.draw_board()

    if computer_starts:
        if start_mode == 'center':
            matrix[1, 1] = computer_icon_representation
        elif start_mode == 'minimax':
            GF.computer_play()
        elif start_mode == 'random':
            x = np.random.randint(3)
            y = np.random.randint(3)
            matrix[x,y] = computer_icon_representation

    GF.display_matrix()

In [14]:
# start_mode:
# 'minimax': 첫 위치 선택 가능
# 'center': 센터에서 시작
# 'random': 랜덤 위치에서 시작
# user_icon:
#  'x' or 'o'
# computer_starts: True or False


start_game(computer_starts = startParam, user_icon = 'x', start_mode = 'random')

points: 0
nodes_visited: 2458
points: 0
nodes_visited: 326
points: 0
nodes_visited: 18
points: 0
nodes_visited: 4
